In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# cd drive/My Drive/ABSA-DistilBERT

/content/drive/My Drive/ABSA-DistilBERT


In [1]:
import pandas as pd
import os
word_to_idx = {"None" : 0, "Positive" : 1, "Negative" : 2}
data_dir = "data/sentihood/bert-pair/"
data = pd.read_csv(os.path.join(data_dir, "train_QA_M.tsv"), sep="\t")
test_data = pd.read_csv(os.path.join(data_dir, "test_QA_M.tsv"), sep="\t")

test_data["label"] = test_data.label.map(word_to_idx)
data["label"] = data.label.map(word_to_idx)

In [2]:
data.head()

,id,sentence1,label,positions
0,0,"LOCATION2 , LOCATION1 general",0,2 3
1,0,"LOCATION2 , LOCATION1 price",0,2 3
2,0,"LOCATION2 , LOCATION1 safety",0,2 3
3,0,"LOCATION2 , LOCATION1 transit location",0,2 3 4
4,1000,LOCATION1 general is one of the most expensive...,0,0 1


In [3]:
no_sentence1_train = 0
no_sentence1_test = 0
no_sentence2_train = 0
no_sentence2_test = 0
positions_train = data.positions
positions_test = test_data.positions
for i, row in enumerate(data.iterrows()):
  pos = int(positions_train[i].split(' ')[0])
  words = row[1].sentence1.split(' ')
  if words[pos] == 'LOCATION1':
    no_sentence1_train += 1
  elif words[pos] == 'LOCATION2':
    no_sentence2_train += 1

for i, row in enumerate(test_data.iterrows()):
  pos = int(positions_test[i].split(' ')[0])
  words = row[1].sentence1.split(' ')
  if words[pos] == 'LOCATION1':
    no_sentence1_test += 1
  elif words[pos] == 'LOCATION2':
    no_sentence2_test += 1

In [4]:
print("Number of SENTENCE1 in training ", no_sentence1_train)
print("Number of SENTENCE2 in training ", no_sentence2_train)
print("Number of SENTENCE1 in testing ", no_sentence1_test)
print("Number of SENTENCE2 in testing ", no_sentence2_test)

Number of SENTENCE1 in training  11712
Number of SENTENCE2 in training  2948
Number of SENTENCE1 in testing  5888
Number of SENTENCE2 in testing  1488


In [5]:
sentences_with_2_locations_train = 0
sentences_with_2_locations_test = 0
for i, row in enumerate(data.iterrows()):
  words = row[1].sentence1.split(' ')
  found = 2
  for word in words:
    if word in 'LOCATION1':
      found -= 1
    elif word in 'LOCATION2':
      found -= 1
  if found == 0:
    sentences_with_2_locations_train += 1

for i, row in enumerate(test_data.iterrows()):
  words = row[1].sentence1.split(' ')
  found = 2
  for word in words:
    if word in 'LOCATION1':
      found -= 1
    elif word in 'LOCATION2':
      found -= 1
  if found == 0:
    sentences_with_2_locations_test += 1

In [6]:
print("No of sentences with 2 locations in training ", sentences_with_2_locations_train)
print("No of sentences with 2 locations in testing ", sentences_with_2_locations_test)

No of sentences with 2 locations in training  6288
No of sentences with 2 locations in testing  3280


In [7]:
max_length_train = -1
min_length_train = 100000
max_length_test = -1
min_length_test = 1000000
for i, row in enumerate(data.iterrows()):
  words = row[1].sentence1.split(' ')
  if(len(words) > max_length_train):
    max_length_train = len(words)
  if(len(words) < min_length_train):
    min_length_train = len(words)


for i, row in enumerate(test_data.iterrows()):
  words = row[1].sentence1.split(' ')
  if(len(words) > max_length_test):
    max_length_test = len(words)
  if(len(words) < min_length_test):
    min_length_test = len(words)

In [8]:
print("Maximum length of a sentence in training ", max_length_train)
print("Minimum length of a sentence in training ", min_length_train)
print("Maximum length of a sentence in testing ", max_length_test)
print("Minimum length of a sentence in testing ", min_length_test)

Maximum length of a sentence in training  166
Minimum length of a sentence in training  3
Maximum length of a sentence in testing  92
Minimum length of a sentence in testing  3


In [9]:
data.groupby('label').count().id

label
0    12548
1     1626
2      834
Name: id, dtype: int64

In [10]:
test_data.groupby('label').count().id

label
0    6300
1     810
2     406
Name: id, dtype: int64

No you don't need position while training on bert